<a href="https://colab.research.google.com/github/sekiguchi-Knowledgecommunication/databricks_api_set_workspace_object/blob/main/6_langchain_handson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. Advanced RAG


In [3]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 83.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
# 【注意】
# 上記の `!pip install numpy==1.26.4` を実行したあと、
# Google Colab 上部のメニューから「ランタイム」の「セッションを再起動する」を実行してください。
# その後このセルを実行して `1.26.4` と表示されることを確認してください。

import numpy as np

print(np.__version__)
assert np.__version__ == "1.26.4"

1.26.4


In [ ]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 \
    langchain-community==0.3.0 GitPython==3.1.43 \
    langchain-chroma==0.1.4 tavily-python==0.5.0 pydantic==2.10.6

In [ ]:
!pip install langchain-cohere==0.3.0

In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

## 6.2. ハンズオンの準備


In [6]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/Stability-AI/stablediffusion",
    repo_path="./stablediffusion",
    branch="main",
    # file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

90


In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("StableDifusionの概要を教えて")

## 6.3. 検索クエリの工夫


### HyDE（Hypothetical Document Embeddings）


In [11]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。

質問: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

In [12]:
hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("Stablediffusionの概要を教えて")

'Stable Diffusionは、テキストから画像を生成するための拡散ベースのモデルです。このモデルは、特に高解像度の画像合成に優れており、Latent Diffusion Model（LDM）として知られています。主な特徴は以下の通りです：\n\n- **開発者**: Robin Rombach、Patrick Esserなど。\n- **モデルタイプ**: テキストから画像への生成モデル。\n- **言語**: 主に英語。\n- **ライセンス**: CreativeML Open RAIL++-M License。\n- **用途**: 研究目的での使用が推奨されており、アート生成、デザイン、教育ツール、生成モデルの研究などに利用されます。\n- **制限**: 完全なフォトリアリズムを達成できず、テキストを正確に描写することが難しい場合があります。また、英語以外の言語での性能が劣ることがあります。\n- **バイアス**: トレーニングデータに基づく社会的バイアスを反映する可能性があり、特に白人や西洋文化がデフォルトとして扱われることが多いです。\n\nStable Diffusionは、特にアートやデザインの分野での創造的なツールとして注目されており、さまざまなバージョン（例：2.0、2.1）が存在し、それぞれ異なる機能や性能を持っています。'

### 複数の検索クエリの生成


In [11]:
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="検索クエリのリスト")


query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。

質問: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

In [12]:
multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("Stable Diffusionの概要を教えて")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


'Stable Diffusionは、テキストから画像を生成するための潜在拡散モデルです。このモデルは、特に画像生成や修正に使用され、ユーザーが提供するテキストプロンプトに基づいて高解像度の画像を生成します。Stable Diffusionは、OpenCLIP-ViT/Hという事前学習済みのテキストエンコーダを使用し、画像を潜在空間にエンコードするオートエンコーダと、拡散モデルを組み合わせています。\n\n主な特徴としては以下があります：\n\n1. **モデルのバージョン**: Stable Diffusionは複数のバージョンがあり、最新のものは2.1です。各バージョンは異なる解像度や機能を持っています。\n\n2. **画像生成**: ユーザーが指定したテキストプロンプトに基づいて、768x768ピクセルの画像を生成します。\n\n3. **画像修正機能**: 画像の修正や拡張、インペインティング（画像の一部を修正する技術）などの機能も提供しています。\n\n4. **バイアスと限界**: モデルは訓練データに基づくバイアスを反映する可能性があり、特に英語以外の言語での性能が劣ることがあります。また、生成される画像は必ずしもフォトリアリズムを達成するわけではありません。\n\n5. **ライセンス**: モデルはCreativeML Open RAIL++-Mライセンスの下で提供されており、研究目的での使用が推奨されています。\n\nStable Diffusionは、アートやデザイン、教育ツールなど、さまざまなクリエイティブな用途に利用されていますが、悪用を避けるためのガードレールを設けることが求められています。'

## 6.4. 検索後の工夫


### RAG Fusion


In [14]:
from langchain_core.documents import Document


def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 各ドキュメントのコンテンツ (文字列) とそのスコアの対応を保持する辞書を準備
    content_score_mapping = {}

    # 検索クエリごとにループ
    for docs in retriever_outputs:
        # 検索結果のドキュメントごとにループ
        for rank, doc in enumerate(docs):
            content = doc.page_content

            # 初めて登場したコンテンツの場合はスコアを0で初期化
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (順位 + k)) のスコアを加算
            content_score_mapping[content] += 1 / (rank + k)

    # スコアの大きい順にソート
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # noqa
    return [content for content, _ in ranked]

In [17]:
rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("Stablediffusionの概要を教えて")

'Stable Diffusionは、テキストから画像を生成するための潜在拡散モデルです。このモデルは、OpenCLIP-ViT/Hテキストエンコーダを使用して、テキストプロンプトに基づいて画像を生成および修正することができます。Stable Diffusionは、特に高解像度の画像合成に優れており、768x768ピクセルの出力を生成します。\n\n主な特徴としては、以下の点が挙げられます：\n\n1. **モデルアーキテクチャ**: Stable Diffusionは、オートエンコーダと拡散モデルを組み合わせた構造を持ち、オートエンコーダの潜在空間で訓練されます。\n\n2. **トレーニングデータ**: LAION-5Bデータセットを使用して訓練されており、成人向けや暴力的なコンテンツを含む可能性がありますが、NSFWフィルタリングが施されています。\n\n3. **利用用途**: アート生成、デザイン、教育ツール、生成モデルの研究など、さまざまな用途に利用されます。ただし、悪用や誤用を避けるためのガードレールを設けることが推奨されています。\n\n4. **バイアスと限界**: モデルは、訓練データに基づくバイアスを反映する可能性があり、特に英語以外の言語での性能が劣ることがあります。また、生成される画像のフォトリアリズムには限界があります。\n\nStable Diffusionは、生成された画像に対して無形のウォーターマークを追加する機能も備えており、機械生成された画像であることを識別する手助けをします。'

### Cohere のリランクモデルを使用する準備


In [2]:
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

### Cohere のリランクモデルの導入


In [15]:
from typing import Any

from langchain_cohere import CohereRerank
from langchain_core.documents import Document


def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
    question = inp["question"]
    documents = inp["documents"]

    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)
    return cohere_reranker.compress_documents(documents=documents, query=question)


rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    | prompt | model | StrOutputParser()
)

rerank_rag_chain.invoke("Stablediffusionの概要を教えて")

'Stable Diffusionは、テキストから画像を生成するための潜在拡散モデルです。このモデルは、CLIP ViT-H/14テキストエンコーダーのペンultimateテキスト埋め込みに条件付けられています。Stable Diffusionは、768x768ピクセルの出力を生成し、さまざまなバージョン（例：2.0、2.1）が存在し、それぞれ異なる機能や解像度を持っています。\n\n主な機能には、画像の生成、画像の修正（img2img）、画像のアップスケーリング、インペインティング（画像の一部を修正すること）などがあります。これらの機能は、特定のモデルや設定を使用して実行され、ユーザーはテキストプロンプトを入力することで画像を生成できます。\n\nStable Diffusionは、オープンソースであり、Hugging Faceを通じてモデルの重みが提供されています。また、モデルは継続的に更新され、新しいチェックポイントが追加される予定です。'

## 6.5. 複数の Retriever を使う工夫


### LLM によるルーティング


In [16]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config(
    {"run_name": "stablediffusion_document_retriever"}
)

web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}
)

In [17]:
from enum import Enum


class Route(str, Enum):
    langchain_document = "langchain_document"
    web = "web"


class RouteOutput(BaseModel):
    route: Route


route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。

質問: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

In [18]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
    question = inp["question"]
    route = inp["route"]

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unknown route: {route}")


route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

In [19]:
route_rag_chain.invoke("Stablediffusionの概要を教えて")

'Stable Diffusionは、Stability AIが開発した画像生成AIで、テキストプロンプトに基づいて高品質な画像を生成することが特徴です。この生成型人工知能モデルは、テキストや画像プロンプトから写真のようにリアルな独自の画像を生成します。初めてリリースされたのは2022年です。アート、デザイン、エンターテインメントなど、さまざまな分野で利用されています。'

In [20]:
route_rag_chain.invoke("東京の今日の天気は？")

'東京の今日、2025年5月30日(金)の天気は、気温が16℃/14℃で、降水確率があるとされています。'

### ハイブリッド検索の実装


In [21]:
!pip install rank-bm25==0.2.2

In [22]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config(
    {"run_name": "chroma_retriever"}
)

bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

In [23]:
from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel({
        "chroma_documents": chroma_retriever,
        "bm25_documents": bm25_retriever,
    })
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

In [24]:
hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)

hybrid_rag_chain.invoke("Stablediffusionの概要を教えて")

'Stable Diffusionは、テキストから画像を生成するための潜在拡散モデルです。このモデルは、CLIP ViT-H/14テキストエンコーダのペンultimate埋め込みに条件付けされており、高解像度の画像を生成する能力を持っています。Stable Diffusionは、さまざまな画像生成タスクに対応しており、特に以下の機能を提供します：\n\n1. **テキストから画像生成**: ユーザーが入力したテキストプロンプトに基づいて画像を生成します。\n2. **画像の修正**: 既存の画像を基に新しい画像を生成する「img2img」機能を提供します。\n3. **画像のアップスケーリング**: 低解像度の画像を高解像度に変換するためのモデルも含まれています。\n4. **インペインティング**: 画像の特定の部分を修正または置き換える機能を持っています。\n\nStable Diffusionは、オープンソースのプロジェクトであり、さまざまなデータセットでトレーニングされており、ユーザーが独自のモデルをトレーニングしたり、既存のモデルを利用したりすることができます。また、生成された画像には、機械生成されたことを示すための目に見えない透かしが追加されることがあります。\n\nこのモデルは、特に高解像度の画像生成において優れた性能を発揮し、さまざまなアプリケーションで利用されています。'